In [59]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.probability import FreqDist

nltk.download('punkt')
nltk.download('stopwords')

# Load the model (adjust the path as necessary)
model_path = '/content/drive/My Drive/BERT_SurveySparrow_Model'
model = BertForSequenceClassification.from_pretrained(model_path)
tokenizer = BertTokenizer.from_pretrained(model_path)
device = torch.device('cpu')
model.to(device)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [60]:
def predict_sentiment(query):
    inputs = tokenizer(query, return_tensors="pt", truncation=True, padding=True, max_length=64)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)

    logits = outputs.logits
    probabilities = torch.softmax(logits, dim=1)
    sentiment_score = probabilities[0][1].item()  # Probability of positive sentiment

    if sentiment_score > 0.6:
        return "Positive"
    elif sentiment_score < 0.4:
        return "Negative"
    else:
        return "Neutral"

def extract_keywords(query, num_keywords=5):
    tokens = word_tokenize(query.lower())
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words and token.isalnum()]
    fdist = FreqDist(tokens)
    return [word for word, _ in fdist.most_common(num_keywords)]

def analyze_query(query):
    sentiment = predict_sentiment(query)
    keywords = extract_keywords(query)
    return {
        "query": query,
        "sentiment": sentiment,
        "keywords": keywords
    }

In [61]:
def generate_response(analysis_result):
    sentiment = analysis_result['sentiment']
    keywords = analysis_result['keywords']

    if sentiment == "Negative":
        return "I'm sorry to hear that you're having difficulties. Let me connect you with a specialist who can help you right away.", True
    elif "password" in keywords:
        return "To reset your password, please go to the login page and click on 'Forgot Password'.", False
    elif "survey" in keywords and "create" in keywords:
        return "Creating a new survey is easy! Just log in to your account and click on 'Create New Survey' on the dashboard.", False
    else:
        return "Thank you for your query. How else can I assist you today?", False


In [80]:
def main():
    print("Welcome to Simple Workspace!")
    print("Type your query or type 'exit' to quit.")

    while True:
        query = input("Your query: ")

        if query.lower() == 'exit':
            print("Exiting...")
            break

        analysis_result = analyze_query(query)
        response, should_escalate = generate_response(analysis_result)

        print("Response:", response)
        if should_escalate:
            print("Escalating call...")
        else:
            print("Not escalating call.")
        print()

if __name__ == "__main__":
    main()


Welcome to Simple Workspace!
Type your query or type 'exit' to quit.
Your query: I forgot my password. is this bad?
Response: To reset your password, please go to the login page and click on 'Forgot Password'.
Not escalating call.

Your query: exit
Exiting...


IVR using Speech to text


In [77]:
import speech_recognition as sr

def audio_to_text(file_path):
    recognizer = sr.Recognizer()

    with sr.AudioFile(file_path) as source:
        audio_data = recognizer.record(source)
        text = recognizer.recognize_google(audio_data)

In [76]:
from pydub import AudioSegment

def process_audio_file(file_path):
    text = audio_to_text(file_path)
    sentiment = predict_sentiment(text)

    if sentiment == "Negative":
        return "I'm sorry to hear that you're having difficulties. Let me connect you with a specialist who can help you right away.", True
    else:
        return "Thank you for your query. How else can I assist you today?", False


In [75]:
def main():
    audio_file_path = '/content/record_out.wav'
    recognized_text = audio_to_text(audio_file_path)
    print("Recognized Text:", recognized_text)

    # Further processing with sentiment analysis or other logic
    sentiment = predict_sentiment(recognized_text)

    if sentiment == "Negative":
        print("Sentiment: Negative")
        print("I'm sorry to hear that you're having difficulties. Let me connect you with a specialist who can help you right away.")
        print("Escalating call...")
    else:
        print("Sentiment: Positive or Neutral")
        print("Thank you for your query. How else can I assist you today?")
        print("Not escalating call.")

if __name__ == "__main__":
    main()



Converted text: the service is too bad
Recognized Text: the service is too bad
Sentiment: Positive or Neutral
Thank you for your query. How else can I assist you today?
Not escalating call.
